In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Lasso
import pickle


In [2]:
def remover_colunas_correlacionadas(df):
    matrix_corr = df.corr().abs()
    remove_columns = set()

    for i_corr in range(len(matrix_corr.columns)):
        for j_corr in range(i_corr + 1, len(matrix_corr.columns)):
            if matrix_corr.iloc[i_corr, j_corr] > 0.87 and matrix_corr.columns[j_corr] != 'close':
                remove_columns.add(matrix_corr.columns[j_corr])

    return df.drop(columns = remove_columns)

def filtra_colunas_zero(df):
    mean = df.describe().loc['mean']
    cols_to_keep = mean[mean != 0].index
    return df[cols_to_keep]

In [3]:
all_stocks = pd.read_csv('data/all_stocks_by_month.csv', sep=',')

FileNotFoundError: [Errno 2] No such file or directory: 'all_stocks_by_month.csv'

In [56]:
all_stocks.head()

,date,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,...,ImpairmentOfCapitalAssets,HedgingAssetsCurrent,CapitalLeaseObligations,CurrentCapitalLeaseObligation,LongTermCapitalLeaseObligation,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,symbol
0,2022-07-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2022-08-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2022-09-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2022-10-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2022-11-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [57]:
all_stocks.columns

Index(['date', 'AccountsPayable', 'AccountsReceivable',
       'AllowanceForDoubtfulAccountsReceivable', 'BasicAverageShares',
       'BasicEPS', 'BeginningCashPosition', 'CapitalStock',
       'CashAndCashEquivalents', 'CashCashEquivalentsAndShortTermInvestments',
       ...
       'ImpairmentOfCapitalAssets', 'HedgingAssetsCurrent',
       'CapitalLeaseObligations', 'CurrentCapitalLeaseObligation',
       'LongTermCapitalLeaseObligation', 'ChangeInInventory',
       'InvestmentsinJointVenturesatCost', 'ChangeInAccruedExpense',
       'DividendPaidCFO', 'symbol'],
      dtype='object', length=242)

In [1]:
def buscar_serie(codigo):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json"
    response = requests.get(url)
    if response.status_code == 200:
        dados = response.json()
        df = pd.DataFrame(dados)
        df['data'] = pd.to_datetime(df['data'], dayfirst=True)
        df['valor'] = df['valor'].astype(float)
        return df
    else:
        print(f"Erro ao acessar a API para o código {codigo}: {response.status_code}")
        return None

def add_economic_indicators():
    indices_economicos = {
        "IPCA": 433,
        "INPC": 188,
        "IGPM": 189,
        "PIB": 4380,
        "DESEMPREGO": 24369,
    }

    dados_indices = {}
    for nome, codigo in indices_economicos.items():
        try:
            dados_indices[nome] = buscar_serie(codigo)
            print(f"Dados de {nome} carregados com sucesso!")
        except Exception as e:
            print(f"Erro ao carregar os dados de {nome}: {e}")

    df_final = None
    for nome, df in dados_indices.items():
        df = df.rename(columns={'valor': nome})
        if df_final is None:
            df_final = df
        else:
            df_final = pd.merge(df_final, df, on='data', how='outer')

    print(df_final.head())
    return df_final

In [3]:
df_indices = add_economic_indicators()
df_indices.rename(columns={'data': 'date'}, inplace=True)
df_indices_cleaned = df_indices.dropna(axis=1, how='all')
df_indices_cleaned.set_index('date', inplace=True)
monthly_indices = df_indices_cleaned.resample('ME').mean()
monthly_indices['date'] = pd.to_datetime(monthly_indices.index)
monthly_indices.reset_index(drop=True, inplace=True)

Dados de IPCA carregados com sucesso!
Dados de INPC carregados com sucesso!
Dados de IGPM carregados com sucesso!
Dados de PIB carregados com sucesso!
Dados de DESEMPREGO carregados com sucesso!
        data  IPCA  INPC  IGPM  PIB  DESEMPREGO
0 1979-05-01   NaN  1.76   NaN  NaN         NaN
1 1979-06-01   NaN  3.00   NaN  NaN         NaN
2 1979-07-01   NaN  5.36   NaN  NaN         NaN
3 1979-08-01   NaN  5.79   NaN  NaN         NaN
4 1979-09-01   NaN  6.61   NaN  NaN         NaN


In [5]:
monthly_indices.to_csv('monthly_indices.csv', index=False)

In [61]:
all_stocks['date']

0       2022-07-31
1       2022-08-31
2       2022-09-30
3       2022-10-31
4       2022-11-30
           ...    
6263    2023-08-31
6264    2023-09-30
6265    2023-10-31
6266    2023-11-30
6267    2023-12-31
Name: date, Length: 6268, dtype: object

In [62]:
def merge_indices_stocks(all_stocks_param, df_indices_cleaned_param):
    # Verificar o formato das datas nos DataFrames
    all_stocks_param['date'] = pd.to_datetime(all_stocks_param['date'])
    df_indices_cleaned_param['date'] = pd.to_datetime(df_indices_cleaned_param['date'])

    # Verificar se há datas comuns entre os DataFrames
    common_dates = set(all_stocks_param['date']).intersection(set(df_indices_cleaned_param['date']))
    if not common_dates:
        print("Não há datas comuns entre os DataFrames.")
    else:
        # Realizar o merge dos DataFrames utilizando a coluna de datas
        merged_df = pd.merge(all_stocks_param, df_indices_cleaned_param, on='date', how='inner')

        # Exibir o DataFrame resultante
        return merged_df


all_stocks = merge_indices_stocks(all_stocks, monthly_indices)
all_stocks

,date,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,...,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,symbol,IPCA,INPC,IGPM,PIB,DESEMPREGO
0,2022-07-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0,-0.68,-0.60,0.21,869133.2,9.1
1,2022-08-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0,-0.36,-0.31,-0.70,873862.3,8.9
2,2022-09-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0,-0.29,-0.32,-0.95,858186.5,8.7
3,2022-10-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0,0.59,0.47,-0.97,880537.5,8.3
4,2022-11-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0,0.41,0.38,-0.56,880820.2,8.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,2023-08-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,ZIFI11.SA,0.23,0.20,-0.14,932293.6,7.8
6264,2023-09-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,ZIFI11.SA,0.26,0.11,0.37,915855.2,7.7
6265,2023-10-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,ZIFI11.SA,0.24,0.12,0.50,953945.0,7.6
6266,2023-11-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,ZIFI11.SA,0.28,0.10,0.59,961832.6,7.5


In [63]:
df_fil = (all_stocks.pipe(filtra_colunas_zero)
               .pipe(remover_colunas_correlacionadas)
               )
df_fil

,date,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,...,InterestReceivedCFO,ImpairmentOfCapitalAssets,HedgingAssetsCurrent,CapitalLeaseObligations,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,IPCA,IGPM
0,2022-07-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.68,0.21
1,2022-08-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.36,-0.70
2,2022-09-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.29,-0.95
3,2022-10-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.59,-0.97
4,2022-11-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41,-0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,2023-08-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.23,-0.14
6264,2023-09-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26,0.37
6265,2023-10-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.50
6266,2023-11-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28,0.59


In [64]:
columns = df_fil.columns

In [65]:
columns.to_list()

['date',
 'AccountsPayable',
 'AccountsReceivable',
 'AllowanceForDoubtfulAccountsReceivable',
 'BasicAverageShares',
 'BasicEPS',
 'BeginningCashPosition',
 'CapitalStock',
 'CashAndCashEquivalents',
 'CashCashEquivalentsAndShortTermInvestments',
 'CashDividendsPaid',
 'CashFinancial',
 'CashFlowsfromusedinOperatingActivitiesDirect',
 'ChangeInCashSupplementalAsReported',
 'ClassesofCashPayments',
 'ClassesofCashReceiptsfromOperatingActivities',
 'CostOfRevenue',
 'CurrentLiabilities',
 'CurrentProvisions',
 'DilutedNIAvailtoComStockholders',
 'DividendsPayable',
 'EBIT',
 'EndCashPosition',
 'EnterpriseValue',
 'FinancingCashFlow',
 'FreeCashFlow',
 'GeneralAndAdministrativeExpense',
 'GrossAccountsReceivable',
 'GrossProfit',
 'InterestIncome',
 'InterestIncomeNonOperating',
 'InterestReceivedDirect',
 'InvestedCapital',
 'InvestmentProperties',
 'LongTermProvisions',
 'MarketCap',
 'NetInterestIncome',
 'NetNonOperatingInterestIncomeExpense',
 'NonCurrentDeferredRevenue',
 'Operati

In [66]:
df_fil = df_fil.dropna()

In [67]:
df_fil.describe()

,date,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,...,InterestReceivedCFO,ImpairmentOfCapitalAssets,HedgingAssetsCurrent,CapitalLeaseObligations,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,IPCA,IGPM
count,6246,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6246.000000,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,...,6.246000e+03,6246.000000,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6246.000000,6.246000e+03,6246.000000,6246.000000
mean,2022-10-10 20:39:11.585014528,4.577433e+06,8.691191e+06,-4.875050e+05,2.525076e+07,20.258868,2.561518e+07,5.622332e+08,7.547121e+06,7.913220e+07,...,2.709350e+04,-376.560999,6.069347e+05,4.390048e+05,5.047198e+04,1.673311e+05,319.884726,-1.149318e+05,0.507707,0.401888
min,2020-06-30 00:00:00,0.000000e+00,0.000000e+00,-4.735000e+07,0.000000e+00,-112.820000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-336000.000000,0.000000e+00,0.000000e+00,-3.800000e+05,0.000000e+00,-22000.000000,-5.982200e+07,-0.680000,-1.930000
25%,2022-01-31 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,5.780000e+05,7.587100e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.240000,-0.520000
50%,2022-10-31 00:00:00,2.470000e+05,0.000000e+00,0.000000e+00,7.584000e+05,0.011344,4.587000e+06,2.154750e+08,5.400000e+04,4.430000e+05,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.530000,0.450000
75%,2023-07-31 00:00:00,2.541000e+06,1.525000e+06,0.000000e+00,4.533315e+06,8.950000,1.980400e+07,6.063280e+08,3.656000e+06,2.417300e+07,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.830000,0.810000
max,2024-06-30 00:00:00,1.829440e+08,2.926470e+08,0.000000e+00,3.924500e+09,4587.040000,6.527920e+08,8.246646e+09,6.679790e+08,3.305701e+09,...,8.722000e+06,336000.000000,4.932320e+08,5.126700e+07,2.535800e+07,1.045150e+08,120000.000000,0.000000e+00,1.620000,4.340000
std,NaN,1.645704e+07,3.382627e+07,3.391159e+06,2.363909e+08,214.836980,6.469390e+07,9.563916e+08,3.489788e+07,2.962395e+08,...,4.520281e+05,27222.213203,1.539059e+07,4.315120e+06,1.040333e+06,4.178924e+06,5730.159425,2.619798e+06,0.448882,1.220795


In [68]:
df_fil

,date,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,...,InterestReceivedCFO,ImpairmentOfCapitalAssets,HedgingAssetsCurrent,CapitalLeaseObligations,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,IPCA,IGPM
0,2022-07-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.68,0.21
1,2022-08-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.36,-0.70
2,2022-09-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.29,-0.95
3,2022-10-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.59,-0.97
4,2022-11-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41,-0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,2023-08-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.23,-0.14
6264,2023-09-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26,0.37
6265,2023-10-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.50
6266,2023-11-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28,0.59


In [69]:
info_target = df_fil[['open', 'close']]
info_target

,open,close
0,95.704001,95.372001
1,91.571305,91.498695
2,90.848095,90.780000
3,91.103499,91.320000
4,92.478000,92.439000
...,...,...
6263,929.541815,923.659995
6264,898.712856,898.855713
6265,897.997998,899.995996
6266,899.992493,897.992493


In [70]:
df_fil.set_index('date', inplace=True)
df_fil = df_fil.drop(columns=['open', 'close'])

In [72]:
# Supondo que 'df_fil' é o seu DataFrame de características
# E 'target' é a coluna alvo para a análise de investimentos
X = df_fil
y = info_target['close']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Obter a importância das características
importances = model.feature_importances_

# Criar um DataFrame com as importâncias das características
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Ordenar as características pela importância
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Selecionar as 15 melhores características
top_15_features = feature_importances.head(15)['Feature'].tolist()

# Exibir as 15 melhores características
print(top_15_features)

['adjclose', 'TotalUnusualItems', 'CostOfRevenue', 'InvestedCapital', 'GrossProfit', 'OrdinarySharesNumber', 'OtherOperatingExpenses', 'FreeCashFlow', 'RetainedEarnings', 'ClassesofCashReceiptsfromOperatingActivities', 'ReceiptsfromCustomers', 'BasicAverageShares', 'TaxEffectOfUnusualItems', 'CashFlowsfromusedinOperatingActivitiesDirect', 'EBIT']


In [73]:
colunas_sugeridas_gpt = [
    "BasicEPS",
    "EBIT",
    "FreeCashFlow",
    "NetIncomeFromContinuingOperations",
    "GrossProfit",
    "CashAndCashEquivalents",
    "MarketCap",
    "EnterpriseValue",
    "TotalLiabilitiesNetMinorityInterest",
    "LongTermDebt",
    "RetainedEarnings",
    "CashFlowsfromusedinOperatingActivitiesDirect",
    "InvestedCapital",
    "AccountsReceivable",
    "dividends",

]

In [74]:
# Salvando
with open("../colunas_extraction/colunas_random_forest_classification.pkl", "wb") as f:
    pickle.dump(top_15_features, f)

# Salvando
with open("../colunas_extraction/colunas_sugeridas_gpt.pkl", "wb") as f:
    pickle.dump(colunas_sugeridas_gpt, f)


In [75]:
# Para ler depois
with open("../colunas_extraction/colunas_random_forest_classification.pkl", "rb") as f:
    feature_RFR = pickle.load(f)
print(feature_RFR)

# Para ler depois
with open("../colunas_extraction/colunas_sugeridas_gpt.pkl", "rb") as f:
    feature_GPT = pickle.load(f)
print(feature_GPT)

['adjclose', 'TotalUnusualItems', 'CostOfRevenue', 'InvestedCapital', 'GrossProfit', 'OrdinarySharesNumber', 'OtherOperatingExpenses', 'FreeCashFlow', 'RetainedEarnings', 'ClassesofCashReceiptsfromOperatingActivities', 'ReceiptsfromCustomers', 'BasicAverageShares', 'TaxEffectOfUnusualItems', 'CashFlowsfromusedinOperatingActivitiesDirect', 'EBIT']
['BasicEPS', 'EBIT', 'FreeCashFlow', 'NetIncomeFromContinuingOperations', 'GrossProfit', 'CashAndCashEquivalents', 'MarketCap', 'EnterpriseValue', 'TotalLiabilitiesNetMinorityInterest', 'LongTermDebt', 'RetainedEarnings', 'CashFlowsfromusedinOperatingActivitiesDirect', 'InvestedCapital', 'AccountsReceivable', 'dividends']


In [76]:
all_stocks_RFR = all_stocks[feature_RFR + ['open', 'close']]
all_stocks_GPT = all_stocks[feature_GPT + ['open', 'close']]

In [77]:
all_stocks_RFR.describe()

,adjclose,TotalUnusualItems,CostOfRevenue,InvestedCapital,GrossProfit,OrdinarySharesNumber,OtherOperatingExpenses,FreeCashFlow,RetainedEarnings,ClassesofCashReceiptsfromOperatingActivities,ReceiptsfromCustomers,BasicAverageShares,TaxEffectOfUnusualItems,CashFlowsfromusedinOperatingActivitiesDirect,EBIT,open,close
count,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6246.000000,6246.000000
mean,-1.138435e+09,1.291482e+07,5.367410e+06,3.468128e+08,2.124772e+07,2.250833e+07,3.162560e+06,2.542556e+07,4.379589e+07,3.795714e+07,1.550227e+07,2.525076e+07,1.923745e+06,2.603182e+07,2.648128e+07,1359.882346,1359.876108
std,6.756262e+10,7.852822e+07,2.434537e+07,6.912260e+08,5.439152e+07,1.030142e+08,1.421019e+07,1.066239e+08,1.691478e+08,9.806552e+07,4.173733e+07,2.363909e+08,9.580953e+06,7.747546e+07,6.305553e+07,13964.254194,13964.255043
min,-4.791499e+12,-2.750930e+08,-1.021500e+07,-1.613900e+07,-5.924300e+07,0.000000e+00,-4.176300e+07,-9.828420e+08,-6.659830e+08,-4.440000e+05,0.000000e+00,0.000000e+00,-3.374738e+07,-1.806550e+08,-7.594700e+07,0.000000,0.000000
25%,4.781842e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.584000e+05,0.000000e+00,-7.850000e+05,-3.619000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.650000e+05,0.000000e+00,62.901125,62.810341
50%,6.596507e+01,0.000000e+00,0.000000e+00,3.776300e+07,0.000000e+00,2.443800e+06,2.075000e+05,6.089000e+06,1.534000e+06,1.232000e+06,0.000000e+00,7.584000e+05,0.000000e+00,0.000000e+00,0.000000e+00,86.702861,86.630952
75%,7.970433e+01,6.104000e+06,7.830000e+05,2.902492e+08,1.276975e+07,1.073410e+07,1.071500e+06,3.649300e+07,3.137900e+07,3.296900e+07,2.520000e+05,4.533315e+06,8.810250e+05,2.299200e+07,1.902300e+07,101.551671,101.583613
max,3.826324e+10,1.224722e+09,2.925620e+08,5.346516e+09,4.670860e+08,1.086780e+09,1.736870e+08,1.103672e+09,1.212509e+09,1.207763e+09,2.811300e+08,3.924500e+09,1.795656e+08,1.103672e+09,5.150330e+08,160000.000000,160000.000000


In [78]:
all_stocks_GPT.describe()

,BasicEPS,EBIT,FreeCashFlow,NetIncomeFromContinuingOperations,GrossProfit,CashAndCashEquivalents,MarketCap,EnterpriseValue,TotalLiabilitiesNetMinorityInterest,LongTermDebt,RetainedEarnings,CashFlowsfromusedinOperatingActivitiesDirect,InvestedCapital,AccountsReceivable,dividends,open,close
count,6246.000000,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6.246000e+03,6246.000000,6246.000000,6246.000000
mean,20.258868,2.648128e+07,2.542556e+07,7.966732e+06,2.124772e+07,7.547121e+06,4.810206e+08,1.623269e+09,8.429658e+07,3.510207e+06,4.379589e+07,2.603182e+07,3.468128e+08,8.691191e+06,0.377946,1359.882346,1359.876108
std,214.836980,6.305553e+07,1.066239e+08,3.752564e+07,5.439152e+07,3.489788e+07,3.104652e+09,2.932816e+10,2.071392e+08,2.343692e+07,1.691478e+08,7.747546e+07,6.912260e+08,3.382627e+07,7.994603,13964.254194,13964.255043
min,-112.820000,-7.594700e+07,-9.828420e+08,-2.173750e+08,-5.924300e+07,0.000000e+00,0.000000e+00,-6.740857e+08,-1.480000e+06,0.000000e+00,-6.659830e+08,-1.806550e+08,-1.613900e+07,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,-7.850000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.344000e+06,0.000000e+00,-3.619000e+06,-1.650000e+05,0.000000e+00,0.000000e+00,0.000000,62.901125,62.810341
50%,0.011344,0.000000e+00,6.089000e+06,0.000000e+00,0.000000e+00,5.400000e+04,7.380000e+07,0.000000e+00,5.446000e+06,0.000000e+00,1.534000e+06,0.000000e+00,3.776300e+07,0.000000e+00,0.024317,86.702861,86.630952
75%,8.950000,1.902300e+07,3.649300e+07,0.000000e+00,1.276975e+07,3.656000e+06,3.762527e+08,1.323280e+08,5.163300e+07,0.000000e+00,3.137900e+07,2.299200e+07,2.902492e+08,1.525000e+06,0.041666,101.551671,101.583613
max,4587.040000,5.150330e+08,1.103672e+09,3.235470e+08,4.670860e+08,6.679790e+08,9.699609e+10,6.663294e+11,1.793707e+09,2.427690e+08,1.212509e+09,1.103672e+09,5.346516e+09,2.926470e+08,388.412457,160000.000000,160000.000000


In [79]:
filtered_stocks_RFR = (all_stocks_RFR.pipe(filtra_colunas_zero)
               .pipe(remover_colunas_correlacionadas)
               )
filtered_stocks_RFR

,adjclose,TotalUnusualItems,CostOfRevenue,InvestedCapital,GrossProfit,OrdinarySharesNumber,OtherOperatingExpenses,FreeCashFlow,RetainedEarnings,ClassesofCashReceiptsfromOperatingActivities,ReceiptsfromCustomers,BasicAverageShares,TaxEffectOfUnusualItems,CashFlowsfromusedinOperatingActivitiesDirect,EBIT,open,close
0,70.472206,211894000.0,0.0,0.0,0.0,14723970.0,0.0,135643000.0,1804000.0,155315000.0,0.0,0.0,0.0,135643000.0,0.0,95.704001,95.372001
1,68.338002,211894000.0,0.0,0.0,0.0,14723970.0,0.0,135643000.0,1804000.0,155315000.0,0.0,0.0,0.0,135643000.0,0.0,91.571305,91.498695
2,68.326817,211894000.0,0.0,0.0,0.0,14723970.0,0.0,135643000.0,1804000.0,155315000.0,0.0,0.0,0.0,135643000.0,0.0,90.848095,90.780000
3,68.847863,211894000.0,0.0,0.0,0.0,14723970.0,0.0,135643000.0,1804000.0,155315000.0,0.0,0.0,0.0,135643000.0,0.0,91.103499,91.320000
4,69.878474,211894000.0,0.0,0.0,0.0,14723970.0,0.0,135643000.0,1804000.0,155315000.0,0.0,0.0,0.0,135643000.0,0.0,92.478000,92.439000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,923.659995,37506000.0,0.0,0.0,0.0,26710000.0,0.0,-374000.0,0.0,0.0,0.0,0.0,12752040.0,0.0,0.0,929.541815,923.659995
6264,898.855713,37506000.0,0.0,0.0,0.0,26710000.0,0.0,-374000.0,0.0,0.0,0.0,0.0,12752040.0,0.0,0.0,898.712856,898.855713
6265,899.995996,37506000.0,0.0,0.0,0.0,26710000.0,0.0,-374000.0,0.0,0.0,0.0,0.0,12752040.0,0.0,0.0,897.997998,899.995996
6266,897.992493,37506000.0,0.0,0.0,0.0,26710000.0,0.0,-374000.0,0.0,0.0,0.0,0.0,12752040.0,0.0,0.0,899.992493,897.992493


In [80]:
filtered_stocks_GPT = (all_stocks_GPT.pipe(filtra_colunas_zero)
               .pipe(remover_colunas_correlacionadas)
               )
filtered_stocks_GPT

,BasicEPS,EBIT,FreeCashFlow,NetIncomeFromContinuingOperations,GrossProfit,CashAndCashEquivalents,MarketCap,EnterpriseValue,TotalLiabilitiesNetMinorityInterest,LongTermDebt,RetainedEarnings,CashFlowsfromusedinOperatingActivitiesDirect,InvestedCapital,AccountsReceivable,dividends,open,close
0,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.000000,95.704001,95.372001
1,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.043478,91.571305,91.498695
2,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.033333,90.848095,90.780000
3,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.007500,91.103499,91.320000
4,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.012500,92.478000,92.439000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,929.541815,923.659995
6264,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,898.712856,898.855713
6265,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,897.997998,899.995996
6266,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,899.992493,897.992493


In [81]:
filtered_stocks_GPT

,BasicEPS,EBIT,FreeCashFlow,NetIncomeFromContinuingOperations,GrossProfit,CashAndCashEquivalents,MarketCap,EnterpriseValue,TotalLiabilitiesNetMinorityInterest,LongTermDebt,RetainedEarnings,CashFlowsfromusedinOperatingActivitiesDirect,InvestedCapital,AccountsReceivable,dividends,open,close
0,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.000000,95.704001,95.372001
1,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.043478,91.571305,91.498695
2,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.033333,90.848095,90.780000
3,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.007500,91.103499,91.320000
4,0.0,0.0,135643000.0,0.0,0.0,80000.0,0.0,0.0,100677000.0,0.0,1804000.0,135643000.0,0.0,0.0,0.012500,92.478000,92.439000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,929.541815,923.659995
6264,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,898.712856,898.855713
6265,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,897.997998,899.995996
6266,0.0,0.0,-374000.0,36898000.0,0.0,3000.0,0.0,0.0,297000.0,0.0,0.0,0.0,0.0,0.0,0.000000,899.992493,897.992493


## Seleção de Características:
Usa Lasso para selecionar características importantes.
Transforma os conjuntos de treino e teste para manter apenas as características selecionadas.
## Validação Cruzada:
Cria um modelo RandomForestRegressor e avalia seu desempenho usando validação cruzada com 5 dobras (cv=5).
Imprime as pontuações de validação cruzada e a média dessas pontuações.
## Treinamento do Modelo:
Treina o modelo RandomForestRegressor com os dados de treino.
## Predição e Avaliação:
Faz previsões no conjunto de teste.
Avalia o modelo usando métricas como MAE (Mean Absolute Error), MSE (Mean Squared Error) e R² (R-squared).
Cria um DataFrame com os valores reais e previstos e exibe as primeiras linhas.

In [85]:
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

imputer = SimpleImputer(strategy='mean')  # Replace NaN with the mean value
filtered_stocks_GPT = filtered_stocks_GPT.copy()
filtered_stocks_GPT[['BasicEPS', 'EBIT', 'FreeCashFlow', 'NetIncomeFromContinuingOperations', 'GrossProfit', 'CashAndCashEquivalents', 'MarketCap', 'EnterpriseValue', 'TotalLiabilitiesNetMinorityInterest', 'LongTermDebt', 'RetainedEarnings', 'CashFlowsfromusedinOperatingActivitiesDirect', 'InvestedCapital', 'AccountsReceivable', 'dividends', 'close']] = imputer.fit_transform(filtered_stocks_GPT[['BasicEPS', 'EBIT', 'FreeCashFlow', 'NetIncomeFromContinuingOperations', 'GrossProfit', 'CashAndCashEquivalents', 'MarketCap', 'EnterpriseValue', 'TotalLiabilitiesNetMinorityInterest', 'LongTermDebt', 'RetainedEarnings', 'CashFlowsfromusedinOperatingActivitiesDirect', 'InvestedCapital', 'AccountsReceivable', 'dividends', 'close']])

dados1 = np.array(filtered_stocks_GPT.drop(columns=['open', 'close']))
dados2 = np.array(filtered_stocks_GPT['close'])

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(dados1, dados2, test_size=0.2, random_state=42)

# Feature Selection usando Lasso
lasso = Lasso(alpha=0.01, max_iter=10000).fit(X_train, y_train)
model = SelectFromModel(lasso, prefit=True)
X_train = model.transform(X_train)
X_test = model.transform(X_test)

# Criar e treinar o modelo Random Forest com validação cruzada
model_regression = RandomForestRegressor(n_estimators=500, random_state=42)
cross_val_scores = cross_val_score(model_regression, X_train, y_train, cv=5)
print(f"Cross-Validation Scores: {cross_val_scores}")
print(f"Mean Cross-Validation Score: {np.mean(cross_val_scores):.4f}")

# Treinar o modelo com os dados de treino
model_regression.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model_regression.predict(X_test)

# Avaliar o modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Criar um DataFrame com os valores reais e previstos
resultados = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# Exibir os primeiros resultados
print(resultados.head())

Cross-Validation Scores: [0.44066169 0.70342335 0.79594348 0.56616173 0.53018911]
Mean Cross-Validation Score: 0.6073
Mean Absolute Error (MAE): 911.1147
Mean Squared Error (MSE): 50794229.0973
R² Score: 0.6421
       Actual   Predicted
0    9.897273   10.597098
1   74.968948  178.543834
2   92.250233  102.860030
3   81.956956   87.441513
4  112.631905  109.301192
